## AntiRef: identity clustering

In [ ]:
# install dependencies
%pip install pandas tqdm abutils

In [ ]:
from collections import Counter
import os
import subprocess as sp
import sys
from typing import Iterable, Optional

from tqdm.notebook import tqdm

import pandas as pd

import abutils

#### build the initial MMSeqs2 database

`mmseqs linclust` requires an MMSeqs2 database as input (not a FASTA file), so we first need to create a database of the filtered input sequences

In [ ]:
def make_mmseqs_db(
    fasta_path: str,
    db_path: str,
    debug: bool = False,
) -> Optional[Iterable[str]]:
    '''
    Builds an MMSeqs2 database from a FASTA-formatted input file.

    Parameters
    ----------
    fasta_path : str
        Path to a FASTA-formatted input file. Required.

    db_path : str
        Path for the MMSeqs2 database. If it does not exist, it will be created. 
        Required.

    debug : bool, default=False
        If ``True``, the ``stdout`` and ``stderr`` from running ``mmseqs`` will be
        returned. If ``False``, nothing is returned. Default is ``False``.
    '''
    fasta_path = os.path.abspath(fasta_path)
    db_path = os.path.abspath(db_path)
    if not os.path.isdir(os.path.dirname(db_path)):
        abutils.io.makedir(os.path.dirname(db_path))
    createdb = f"mmseqs createdb {fasta_path} {db_path}"
    p = sp.Popen(createdb, shell=True, stdout=sp.PIPE, stderr=sp.PIPE)
    stdout, stderr = p.communicate()
    if debug:
        return stdout, stderr

In [ ]:
init_fasta = './data/filtered/fasta/all.fasta'
init_db = './data/antiref/seqdb/init'

make_mmseqs_db(
    fasta_path=init_fasta,
    db_path=init_db,
)

### cluster AntiRef datasets




In [ ]:
def linclust(
    threshold: int,
    parent_db: str,
    temp_dir: str = '/tmp',
    debug: bool = False,
) -> Optional[Iterable[str]]:
    '''
    
    '''
    # set up file and DB paths
    temp_dir = os.path.abspath(temp_dir)
    parent_db = os.path.abspath(parent_db)
    cluster_db = f"./data/antiref/clusterdb/antiref{threshold}"
    seq_db = f"./data/antiref/seqdb/antiref{threshold}"
    seq_tsv = f"./data/antiref/tsv/antiref{threshold}.tsv"
    cluster_sizes = f"./data/antiref/tsv/antiref{threshold}_cluster-sizes.csv"
    seq_fasta = f"./data/antiref/fasta/antiref{threshold}.fasta"
    for path in [cluster_db, seq_db, seq_tsv, seq_fasta]:
        d = os.path.dirname(path)
        if not os.path.isdir(d):
            abutils.io.make_dir(d)
    # do the clustering
    cmd = f"mmseqs linclust {parent_db} {cluster_db} {temp_dir} "
    cmd += "--cov-mode 0 "
    cmd += "-c 1.0 "
    cmd += f"--min-seq-id {threshold/100:0.2f} "
    cmd += f"&& mmseqs createsubdb {cluster_db} {parent_db} {seq_db} "
    cmd += f"&& mmseqs convert2fasta {seq_db} {seq_fasta} "
    cmd += f"&& mmseqs createtsv {parent_db} {parent_db} {cluster_db} {seq_tsv} "
    cmd += f"&& cut -f 1 {seq_tsv} | sort | uniq -c > {cluster_sizes} "
    if debug:
        print('')
        print(cmd)
        print('')
    p = sp.Popen(cmd, shell=True, stdout=sp.PIPE, stderr=sp.PIPE)
    stdout, stderr = p.communicate()
    if debug:
        return stdout, stderr

In [ ]:
thresholds = [100, 99, 98, 96, 94, 92, 90]
parent_lookup = {
    100: "./data/antiref/seqdb/init",
    99: "./data/antiref/seqdb/antiref100",
    98: "./data/antiref/seqdb/antiref99",
    96: "./data/antiref/seqdb/antiref98",
    94: "./data/antiref/seqdb/antiref96",
    92: "./data/antiref/seqdb/antiref94",
    90: "./data/antiref/seqdb/antiref92",
}

In [ ]:
for threshold in thresholds:
    parent_db = parent_lookup[threshold]
    print(f'clustering at {threshold/100:0.2f} identity using linclust...')
    print(f'  - parent: {parent_db}')
    sys.stdout.flush()
    linclust(
        threshold=threshold,
        parent_db=parent_db,
    )

### cluster manifest

The cluster manifest is a CSV file of the format:

| antiref100 | antiref99 | antiref98 | antiref96 | antiref04 | antiref92 | antiref90 |
| --- | --- | --- | --- | --- | --- | --- |
| id_01 | id_01 | id_01 | id_01 | id_01 | id_01 | id_03 |
| id_02 | id_01 | id_01 | id_01 | id_01 | id_01 | id_03 |
| id_03 | id_03 | id_03 | id_03 | id_03 | id_03 | id_03 |
| id_04 | id_04 | id_04 | id_04 | id_04 | id_04 | id_04 |
| id_05 | id_05 | id_05 | id_05 | id_05 | id_05 | id_05 |

To interpret the cluster manifest, it's important to remember that we perform nested clustering, and that cluster names are taken from the representative sequence in the cluster (as determined by `mmseqs`). Walking through this mock data, we observe several things:

1. The sequences `id_01` and `id_02` are at least 99% similar but not identical, as they are separate in AntiRef100 but merge in AntiRef99
2. `id_01` becomes the representative sequence in the AntiRef99 cluster containing `id_01` and `id_02` 
3. The sequence `id_03` is 90-92% similar to `id_01` and `id_02` and becomes the representative of the resulting AntiRef90 cluster
4. The sequences `id_04` and `id_05` are less than 90% similar to each other and to `id_01`, `id_02`, and `id_03`.

The manifest can also be used to infer the relative diversity of any AntiRef cluster. If we filter AntiRef90 cluster `id_03`, we can quickly determine that two of the three sequences are nearly identical. Extrapolating this, an AntiRef90 cluster containing 100 sequences of which 99 clustered together in AntiRef99 is much less diverse than the same size AntiRef90 cluster for which none of the sequences clustered together in of the higher identity AntiRef clusters.

To create the cluster manifest, we first need to iterate through all of the AntiRef datasets and parse the cluster assignments. The result is a single nested dictionary with results for all AntiRef datasets:

In [ ]:
cluster_dicts = {}
pbar = tqdm(thresholds)

for threshold in pbar:
    d = {}
    pbar.set_description(f"AntiRef{threshold}")
    with open(f"./data/antiref/tsv/antiref{threshold}.tsv") as f:
        for line in f:
            if l := line.strip().split():
                d[l[1]] = l[0]
    cluster_dicts[threshold] = d

We can then and write the results as a CSV-formatted file:

In [ ]:
manifest_csv = './data/antiref/antiref_cluster-manifest.csv'
if not os.path.isdir(os.path.dirname(manifest_csv)):
    abutils.io.make_dir(os.path.dirname(manifest_csv))

with open(manifest_csv, 'w') as f:
    header = [f'antiref{c}' for c in thresholds]
    f.write(','.join(header) + '\n')
    all_names = list(cluster_dicts[99].keys())
    for c100 in tqdm(all_names):
        c99 = cluster_dicts[99][c100]
        c98 = cluster_dicts[98][c99]
        c96 = cluster_dicts[96][c98]
        c94 = cluster_dicts[94][c96]
        c92 = cluster_dicts[92][c94]
        c90 = cluster_dicts[90][c92]
        f.write(','.join([c100, c99, c98, c96, c94, c92, c90]) + '\n')

### clustering efficiency

Finally, we'll quantify dataset compression for each AntiRef dataset. The result is a CSV-formatted file of the format:

| clustering_identity | clusters | efficiency |
| --- | --- | --- |
| 100 | 100,000,000 | 1.00 |
| 99 | 90,000,000 | 0.90 |
| 98 | 88,000,000 | 0.88 |

The efficiency is computed relative to AntiRef100, which contains all of the unique sequences from the filtered input data.

In [ ]:
eff_path = './data/antiref/clustering_efficiencies.csv'
if not os.path.isdir(os.path.dirname(eff_path)):
    abutils.io.make_dir(os.path.dirname(eff_path))

eff_data = []
antiref100_count = None

pbar = tqdm(thresholds)
for threshold in pbar:
    pbar.set_description(f"AntiRef{threshold}")
    size_path = os.path.abspath(f"./data/antiref/tsv/antiref{threshold}_cluster-sizes.csv")

    # find the dataset size by counting the number lines in the cluster_sizes CSV file
    wc = f"wc -l {size_path}"
    p = sp.Popen(wc, shell=True, stdout=sp.PIPE, stderr=sp.PIPE)
    stdout, _ = p.communicate()
    count = int(stdout.decode('utf-8').split()[0])

    # set AntiRef100 count for dataset size normalization
    if threshold == 100:
        antiref100_count = count
    
    eff_data.append({'clustering_identity': threshold,
                     'clusters': count,
                     'efficiency': count / antiref100_count})

# save efficiency data
eff_df = pd.DataFrame(eff_data)
eff_df.to_csv(eff_path, index=False)